## GOAL OF PROJECT

To implement the following paper "Predicting the direction of stock market prices using random forest" - Luckyson Khaidem, Snehanshu Saha, Sudeepa Roy Dey

### IMPORTING THE LIBRARIES REQUIRED FOR THE TASK 

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

## The model of choice for training and prediction purposes is the Random Forest Classifier

### Importing required libraries for training and testing the model

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, confusion_matrix, recall_score, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import TimeSeriesSplit

### In order to use the technical financial indicators required in the paper, we utilise a pre-defined library as follows:

## https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py

#### Importing the above library

In [3]:
import pandas_techinal_indicators as ta 

## Data

For our purpose of implementing the paper, we are using closing stock price of RELIANCE which has been obtained from 
NSE dataset

In [4]:
raw_data = pd.read_csv('RELIANCE.csv')
del(raw_data['Series'])
del(raw_data['Date'])
del(raw_data['Symbol'])
del(raw_data['N'])


## Exponential Smoothing 

As indicated by the authors of the paper, this is done for the purpose of putting more importance on recent data and exponentially decreasing weightage to past data 

In [5]:
# Function for exponentially smoothing

def exp_smoothing(df, alpha):
    es_data = df.ewm(alpha=alpha).mean()    
    return es_data

In [6]:
# For current testing purposes, value of alpha used is 0.9

sdata = exp_smoothing(raw_data, 0.9)

# Let us visualise the data

sdata.head() 

,Prev Close,Open,High,Low,Last,Close,Average,Volume,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,891.150000,886.300000,892.000000,883.650000,888.000000,887.900000,887.620000,6.774390e+05,6.013059e+08,16263.000000,2.863770e+05,42.270000
1,888.195455,887.936364,895.681818,884.240909,885.227273,885.763636,889.229091,1.585064e+06,1.409629e+09,29904.818182,9.792552e+05,60.724545
2,885.812162,885.290991,891.373874,875.104955,876.509009,876.832432,882.608288,2.237791e+06,1.974631e+09,63355.432432,1.352117e+06,60.360090
3,876.845320,871.527723,874.835734,836.306616,836.896940,840.169577,851.097678,4.056200e+06,3.445830e+09,115031.010801,2.253248e+06,55.778551
4,840.174165,840.542462,860.403429,836.930668,855.889884,852.887085,849.938756,4.668310e+06,3.967063e+09,124848.299253,2.822252e+06,60.405901


## Technical indicators used for Feature Extraction


In [7]:
def feature_extraction(data):
    for x in [5, 14, 26, 44, 66]:
        data = ta.relative_strength_index(data, n=x)
        data = ta.stochastic_oscillator_d(data, n=x)
        data = ta.accumulation_distribution(data, n=x)
        data = ta.average_true_range(data, n=x)
        data = ta.momentum(data, n=x)
        data = ta.money_flow_index(data, n=x)
        data = ta.rate_of_change(data, n=x)
        data = ta.on_balance_volume(data, n=x)
        data = ta.commodity_channel_index(data, n=x)
        data = ta.ease_of_movement(data, n=x)
        data = ta.trix(data, n=x)
        data = ta.vortex_indicator(data, n=x)
    
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema14'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()
    
    data = ta.macd(data, n_fast=12, n_slow=26)
    
    del(data['Open'])
    del(data['Prev Close'])
    del(data['High'])
    del(data['Low'])
    del(data['Volume'])
    del(data['Last'])
    del(data['Average'])
    del(data['Turnover'])
    del(data['No. of Trades'])
    del(data['Deliverable Qty'])
    del(data['% Dly Qt to Traded Qty'])
    
    return data
   
def compute_prediction_int(df, n):
    pred = (df.shift(-n)['Close'] >= df['Close'])
    pred = pred.iloc[:-n]
    return pred.astype(int)

def prepare_data(df, horizon):
    data = feature_extraction(df).dropna().iloc[:-horizon]
    data['pred'] = compute_prediction_int(data, n=horizon)
    del(data['Close'])
    return data.dropna()

## Preparation of training data and labels.

### Assume a prediction horizon of 1 day

In [8]:
data = prepare_data(sdata, 30)

## Identifying and extracting the label
y = data['pred']

## Extracting the input features and creating the input feature matrix
input_feature = [x for x in data.columns if x not in ['gain', 'pred']]
X = data[input_feature]
data

,RSI_5,SO%d_5,Acc/Dist_ROC_5,ATR_5,Momentum_5,MFI_5,ROC_5,OBV_5,CCI_5,EoM_5,...,Trix_66,Vortex_66,ema50,ema21,ema14,ema5,MACD_12_26,MACDsign_12_26,MACDdiff_12_26,pred
196,0.709959,0.608473,-1.663289,16.130333,-10.108233,0.4,0.011883,1.106131e+04,1.315425,-0.000017,...,-0.000259,-0.113691,0.994980,1.013783,1.022121,1.017671,3.836370,-2.182139,6.018509,1.0
197,0.810461,0.532106,-0.570848,14.400149,12.129177,0.6,0.017769,9.147148e+05,1.449906,-0.000005,...,-0.000273,-0.114298,0.996741,1.014795,1.022237,1.016118,5.010137,-0.743684,5.753821,1.0
198,0.858755,0.595927,-3.163291,13.294758,23.532918,0.8,0.022368,2.016210e+06,1.308485,0.000012,...,-0.000285,-0.109536,1.005194,1.022416,1.028887,1.020618,6.491585,0.703370,5.788215,1.0
199,0.973660,0.700299,7.653057,25.228926,66.928292,0.8,0.072650,3.162677e+06,1.714014,0.000037,...,-0.000295,-0.088357,1.055823,1.071477,1.076064,1.060302,11.326843,2.828065,8.498779,1.0
200,0.977109,0.590569,-2.048099,21.701231,63.482829,1.0,0.060827,1.990293e+06,1.193461,0.000052,...,-0.000301,-0.090686,1.053116,1.066500,1.069160,1.048468,14.872721,5.236996,9.635725,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,0.856570,0.666886,0.592677,31.112212,50.251418,0.8,0.035220,4.281019e+06,1.054439,0.000021,...,0.001403,-0.009037,1.018790,1.000470,1.005425,1.018019,-15.002661,-21.929542,6.926881,0.0
1271,0.882526,0.512705,2.339382,31.799127,43.410142,1.0,0.019301,4.372975e+06,1.145540,0.000046,...,0.001380,-0.003940,1.013611,0.995852,1.000547,1.010909,-12.392419,-20.022117,7.629699,0.0
1272,0.562977,0.503365,-1.055666,27.918790,17.526014,0.8,0.000130,2.042255e+06,-0.630862,0.000014,...,0.001356,-0.020669,1.006302,0.989299,0.993891,1.003120,-11.042164,-18.226127,7.183962,0.0
1273,0.782382,0.631243,0.525527,30.321166,31.902601,0.8,0.018122,2.063026e+06,1.307338,0.000026,...,0.001333,-0.018140,1.027467,1.010173,1.014315,1.020566,-7.328706,-16.046643,8.717936,0.0


### Scaling the data into range (-1,1) for pre-processing  

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_new = scaler.fit_transform(X)

print("This is X_new: ")
print(X_new)

This is X_new: 
[[0.70925092 0.65990278 0.87400657 ... 0.77150417 0.7446116  0.78361608]
 [0.81008762 0.54744159 0.87562356 ... 0.77590546 0.75071226 0.78155855]
 [0.85854186 0.64142722 0.87178631 ... 0.78146046 0.75684939 0.78182591]
 ...
 [0.56178057 0.50511661 0.87490595 ... 0.71571394 0.67656711 0.79267558]
 [0.7819151  0.69343538 0.87724638 ... 0.72963834 0.68581056 0.80459974]
 [0.80489093 0.4749087  0.87323745 ... 0.73700412 0.69487154 0.80326193]]


## HERE, instead of splitting into training and testing radnomly (as is done usually), we are using the TimeSeriesSplit method to split the data into training and testing data

### This is because, since we are using time series data, random splitting may lead to data leakage

In [10]:
tscv = TimeSeriesSplit()

for train_index, test_index in tscv.split(X_new):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_new[train_index, :], X_new[test_index,:]
    
for train_index, test_index in tscv.split(y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

## Printing sizes to verify correctness
print('len X_train', len(X_train))
print('len y_train', len(y_train))
print('len X_test', len(X_test))
print('len y_test', len(y_test))

len X_train 900
len y_train 900
len X_test 179
len y_test 179


## MODEL - Random Forests 

In [11]:
model = RandomForestClassifier(n_jobs=-1, n_estimators=70, random_state=42)


## Training and testing the model

In [12]:
model.fit(X_train, y_train.values.ravel());

prediction = model.predict(X_test)

accuracy = accuracy_score(y_pred=prediction, y_true=y_test)

print('Accuracy: {0:1.2f}'.format(accuracy))


confusion = confusion_matrix(y_pred=prediction, y_true=y_test)
print('Confusion Matrix')
print(confusion)


precision = precision_score(y_pred=prediction, y_true=y_test)
recall = recall_score(y_pred=prediction, y_true=y_test)
f1 = f1_score(y_pred=prediction, y_true=y_test)
print('Precision: {0:1.2f}, Recall: {1:1.2f}, f1: {2:1.2f}'.format(precision, recall, f1))


Accuracy: 0.50
Confusion Matrix
[[45 56]
 [33 45]]
Precision: 0.45, Recall: 0.58, f1: 0.50
